In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import csv

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the generator network
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(2, 16),  # Input size is 2 (latent space)
            nn.LeakyReLU(0.2),
            nn.Linear(16, 16),
            nn.LeakyReLU(0.2),
            nn.Linear(16, 2)  # Output size should be 2 for 2D data
        )

    def forward(self, input):
        return self.main(input)

# Define the discriminator network
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(2, 16),  # Input size should match the generated data size
            nn.LeakyReLU(0.2),
            nn.Linear(16, 16),
            nn.LeakyReLU(0.2),
            nn.Linear(16, 1)  # Output size is 1 for binary classification
        )

    def forward(self, input):
        return self.main(input)

# Create instances of the generator and discriminator and move to device
generator = Generator().to(device)
discriminator = Discriminator().to(device)

# Loss function
criterion = nn.BCEWithLogitsLoss()

# Optimizers
gen_optimizer = optim.RMSprop(generator.parameters(), lr=0.001)
disc_optimizer = optim.RMSprop(discriminator.parameters(), lr=0.001)

# Function to generate noise
def sample_Z(m, n):
    return torch.Tensor(np.random.uniform(-1., 1., size=[m, n])).to(device)

# Load training data from m_hist.txt
def load_data(file_path):
    # Load only the first two columns to avoid dimension mismatch
    data = np.loadtxt(file_path, usecols=[0, 1])  
    return torch.Tensor(data).to(device)

# Load your real training data
real_data_file = '/home/jovyan/plots-gan/basic-gans/code/m_hist.txt'  # Full path to your data file
x_plot = load_data(real_data_file)

# Training parameters
batch_size = min(256, len(x_plot))  # Adjust batch size to be smaller than or equal to the number of data points
nd_steps = 10
ng_steps = 10
num_iterations = 12001  # Set to 12000 iterations

# Path to the loss log file
loss_log_file = '../plots/loss_log.csv'

# Write the header of the CSV file
with open(loss_log_file, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['Iteration', 'Discriminator Loss', 'Generator Loss'])

# Training loop
for i in range(num_iterations):
    for _ in range(nd_steps):
        # Sample a batch of real data
        indices = np.random.choice(len(x_plot), batch_size, replace=False)
        X_batch = x_plot[indices]
        Z_batch = sample_Z(batch_size, 2)

        # Real data labels are 1, generated data labels are 0
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # Compute discriminator loss on real data
        real_outputs = discriminator(X_batch)
        real_loss = criterion(real_outputs, real_labels)

        # Compute discriminator loss on fake data
        fake_data = generator(Z_batch)
        fake_outputs = discriminator(fake_data.detach())
        fake_loss = criterion(fake_outputs, fake_labels)

        # Total discriminator loss
        disc_loss = real_loss + fake_loss

        # Backpropagation and optimization for discriminator
        disc_optimizer.zero_grad()
        disc_loss.backward()
        disc_optimizer.step()

    for _ in range(ng_steps):
        Z_batch = sample_Z(batch_size, 2)
        fake_data = generator(Z_batch)
        fake_outputs = discriminator(fake_data)

        # Generator loss
        gen_loss = criterion(fake_outputs, real_labels)

        # Backpropagation and optimization for generator
        gen_optimizer.zero_grad()
        gen_loss.backward()
        gen_optimizer.step()

    # Save the losses to the CSV file
    with open(loss_log_file, 'a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([i, disc_loss.item(), gen_loss.item()])

    print(f"Iterations: {i}\t Discriminator loss: {disc_loss.item():.4f}\t Generator loss: {gen_loss.item():.4f}")

    # Plotting every 1000 iterations
    if i % 1000 == 0:
        plt.figure()
        with torch.no_grad():
            g_plot = generator(sample_Z(batch_size, 2)).cpu().numpy()  # Generate new data for plotting
        plt.scatter(x_plot.cpu()[:, 0], x_plot.cpu()[:, 1], color='blue', label='Real Data', alpha=0.6)
        plt.scatter(g_plot[:, 0], g_plot[:, 1], color='orange', label='Generated Data', alpha=0.6)

        plt.legend()
        plt.title(f'Samples at Iteration {i}', fontsize=14)
        plt.xlabel('X-axis Label', fontsize=12)
        plt.ylabel('Y-axis Label', fontsize=12)
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(f'../plots/iterations/iteration_{i}.png', dpi=300)
        plt.close()


FileNotFoundError: /home/jovyan/plots-gan/basic-gans/code/m_hist.txt not found.

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import matplotlib.pyplot as plt
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Hyperparameters ---
latent_dim = 32  # latent space dimension
hidden_dim = 128  # neurons per hidden layer
batch_size = 128
num_iterations = 30000
lr = 0.0002
beta1, beta2 = 0.5, 0.999  # Adam betas

# --- Generator ---
class Generator(nn.Module):
    def __init__(self, latent_dim, output_dim):
        super(Generator, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.LeakyReLU(0.2),

            nn.Linear(hidden_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.LeakyReLU(0.2),

            nn.Linear(hidden_dim, output_dim),
            # No activation here if output can have negative or large values
        )

    def forward(self, z):
        return self.net(z)

# --- Discriminator ---
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.LeakyReLU(0.2),

            nn.Linear(hidden_dim, hidden_dim),
            nn.LeakyReLU(0.2),

            nn.Linear(hidden_dim, 1)
            # Output logits (no sigmoid here because we use BCEWithLogitsLoss)
        )

    def forward(self, x):
        return self.net(x)

# --- Load and Prepare Data ---
def load_data(file_path):
    data = np.loadtxt(file_path, delimiter=',')
    data_tensor = torch.tensor(data, dtype=torch.float32)
    # Normalize data between -1 and 1 for stable training
    min_val = data_tensor.min()
    max_val = data_tensor.max()
    data_tensor = 2 * (data_tensor - min_val) / (max_val - min_val) - 1
    return data_tensor.to(device), min_val, max_val

real_data_file = '/home/jovyan/Muon_decay_GAN/GAN/code/m_hist.txt'
real_data, data_min, data_max = load_data(real_data_file)

dataset = TensorDataset(real_data)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)

data_dim = real_data.shape[1]  # output dimension

# --- Instantiate models ---
generator = Generator(latent_dim, data_dim).to(device)
discriminator = Discriminator(data_dim).to(device)

# --- Optimizers ---
gen_optimizer = optim.Adam(generator.parameters(), lr=lr, betas=(beta1, beta2))
disc_optimizer = optim.Adam(discriminator.parameters(), lr=lr, betas=(beta1, beta2))

# --- Loss ---
criterion = nn.BCEWithLogitsLoss()

# --- Training Loop ---
for iteration in range(num_iterations):
    for real_batch_tuple in dataloader:
        real_batch = real_batch_tuple[0].to(device)

        batch_size_curr = real_batch.size(0)

        # Labels with smoothing
        real_labels = torch.full((batch_size_curr, 1), 0.9, device=device)
        fake_labels = torch.zeros((batch_size_curr, 1), device=device)

        # --- Train Discriminator ---
        disc_optimizer.zero_grad()

        # Real data
        real_preds = discriminator(real_batch)
        real_loss = criterion(real_preds, real_labels)

        # Fake data
        z = torch.randn(batch_size_curr, latent_dim, device=device)
        fake_data = generator(z)
        fake_preds = discriminator(fake_data.detach())
        fake_loss = criterion(fake_preds, fake_labels)

        disc_loss = real_loss + fake_loss
        disc_loss.backward()
        disc_optimizer.step()

        # --- Train Generator ---
        gen_optimizer.zero_grad()

        z = torch.randn(batch_size_curr, latent_dim, device=device)
        fake_data = generator(z)
        fake_preds = discriminator(fake_data)
        # We want generator to fool discriminator, so labels are real
        gen_loss = criterion(fake_preds, real_labels)

        gen_loss.backward()
        gen_optimizer.step()

    if iteration % 500 == 0:
        print(f"Iter {iteration} | Disc Loss: {disc_loss.item():.4f} | Gen Loss: {gen_loss.item():.4f}")

        # Generate some samples for visualization
        generator.eval()
        with torch.no_grad():
            z = torch.randn(batch_size, latent_dim, device=device)
            generated_samples = generator(z).cpu()

            # Rescale to original range for plotting
            generated_samples = (generated_samples + 1) / 2  # scale back to [0,1]
            generated_samples = generated_samples * (data_max - data_min) + data_min

        plt.figure(figsize=(6, 6))
        plt.scatter(real_data.cpu()[:, 0], real_data.cpu()[:, 1], alpha=0.3, label="Real Data")
        plt.scatter(generated_samples[:, 0], generated_samples[:, 1], alpha=0.3, label="Generated Data")
        plt.legend()
        plt.title(f"Flux GAN Samples at Iter {iteration}")
        plt.show()
        generator.train()


ValueError: could not convert string '76.2012196342366 6.6705621039015695' to float64 at row 0, column 1.

'/home/jovyan/Muon_decay_GAN/GAN/code'

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import csv
import os

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Generator Network
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(2, 16),
            nn.LeakyReLU(0.2),
            nn.Linear(16, 16),
            nn.LeakyReLU(0.2),
            nn.Linear(16, 2),
        )

    def forward(self, input):
        return self.main(input)

# Discriminator Network
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(2, 16),
            nn.LeakyReLU(0.2),
            nn.Linear(16, 16),
            nn.LeakyReLU(0.2),
            nn.Linear(16, 1),
        )

    def forward(self, input):
        return self.main(input)

# Initialize models and move to device
generator = Generator().to(device)
discriminator = Discriminator().to(device)

# Loss and optimizers
criterion = nn.BCEWithLogitsLoss()
gen_optimizer = optim.RMSprop(generator.parameters(), lr=0.001)
disc_optimizer = optim.RMSprop(discriminator.parameters(), lr=0.001)

# Noise sampling
def sample_Z(m, n):
    return torch.Tensor(np.random.uniform(-1., 1., size=[m, n])).to(device)

# Load your nuclear flux data (adjust path as needed)
def load_data(file_path):
    data = np.loadtxt(file_path, usecols=[0,1])
    return torch.Tensor(data).to(device)

real_data_file = '/home/jovyan/Muon_decay_GAN/GAN/code/m_hist.txt'
x_plot = load_data(real_data_file)

# Training params
batch_size = min(256, len(x_plot))
nd_steps = 10
ng_steps = 10
num_iterations = 100001

# Setup directories
log_dir = '../plots'
checkpoint_dir = os.path.join(log_dir, 'checkpoint')
plot_dir = os.path.join(log_dir, 'iterations')
os.makedirs(checkpoint_dir, exist_ok=True)
os.makedirs(plot_dir, exist_ok=True)

loss_log_file = os.path.join(log_dir, 'loss_log.csv')

# Checkpoint saving
def save_checkpoint(iteration):
    path = os.path.join(checkpoint_dir, f'checkpoint_{iteration}.tar')
    torch.save({
        'iteration': iteration,
        'generator_state_dict': generator.state_dict(),
        'discriminator_state_dict': discriminator.state_dict(),
        'gen_optimizer_state_dict': gen_optimizer.state_dict(),
        'disc_optimizer_state_dict': disc_optimizer.state_dict(),
    }, path)
    print(f"[Checkpoint] Saved at iteration {iteration}")

# Checkpoint loading
def load_checkpoint():
    files = [f for f in os.listdir(checkpoint_dir) if f.endswith('.tar')]
    if not files:
        print("[Checkpoint] No checkpoint found. Starting fresh.")
        return 0
    latest = max(files, key=lambda f: int(f.split('_')[1].split('.')[0]))
    path = os.path.join(checkpoint_dir, latest)
    checkpoint = torch.load(path)
    generator.load_state_dict(checkpoint['generator_state_dict'])
    discriminator.load_state_dict(checkpoint['discriminator_state_dict'])
    gen_optimizer.load_state_dict(checkpoint['gen_optimizer_state_dict'])
    disc_optimizer.load_state_dict(checkpoint['disc_optimizer_state_dict'])
    print(f"[Checkpoint] Loaded from iteration {checkpoint['iteration']}")
    return checkpoint['iteration']

# Load checkpoint if exists
start_iteration = load_checkpoint()

# Initialize CSV log file if fresh start
if start_iteration == 0:
    with open(loss_log_file, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Iteration', 'Discriminator Loss', 'Generator Loss'])

# Training loop
for i in range(start_iteration, num_iterations):
    for _ in range(nd_steps):
        indices = np.random.choice(len(x_plot), batch_size, replace=False)
        real_batch = x_plot[indices]
        noise_batch = sample_Z(batch_size, 2)

        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # Discriminator real loss
        real_out = discriminator(real_batch)
        real_loss = criterion(real_out, real_labels)

        # Discriminator fake loss
        fake_data = generator(noise_batch).detach()
        fake_out = discriminator(fake_data)
        fake_loss = criterion(fake_out, fake_labels)

        disc_loss = real_loss + fake_loss

        disc_optimizer.zero_grad()
        disc_loss.backward()
        disc_optimizer.step()

    for _ in range(ng_steps):
        noise_batch = sample_Z(batch_size, 2)
        fake_data = generator(noise_batch)
        fake_out = discriminator(fake_data)
        gen_loss = criterion(fake_out, real_labels)

        gen_optimizer.zero_grad()
        gen_loss.backward()
        gen_optimizer.step()

    # Log losses
    with open(loss_log_file, 'a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([i, disc_loss.item(), gen_loss.item()])

    if i % 100 == 0 or i == start_iteration:
        print(f"Iteration {i}: Disc Loss={disc_loss.item():.4f}, Gen Loss={gen_loss.item():.4f}")

    # Save checkpoint every 1000 iterations
    if i % 1000 == 0 and i != start_iteration:
        save_checkpoint(i)

    # Save plot every 1000 iterations
    if i % 1000 == 0:
        with torch.no_grad():
            generated_samples = generator(sample_Z(batch_size, 2)).cpu().numpy()
            real_samples = x_plot.cpu().numpy()

        plt.figure(figsize=(6,6))
        plt.scatter(real_samples[:, 0], real_samples[:, 1], color='blue', alpha=0.6, label='Real Data')
        plt.scatter(generated_samples[:, 0], generated_samples[:, 1], color='orange', alpha=0.6, label='Generated Data')
        plt.legend()
        plt.title(f"GAN Flux Samples at Iteration {i}")
        plt.xlabel('X')
        plt.ylabel('Y')
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(os.path.join(plot_dir, f'iteration_{i}.png'), dpi=300)
        plt.close()


[Checkpoint] Loaded from iteration 95000
Iteration 95000: Disc Loss=1.2610, Gen Loss=0.5782
Iteration 95100: Disc Loss=1.2679, Gen Loss=0.6016
Iteration 95200: Disc Loss=1.2953, Gen Loss=0.6222
Iteration 95300: Disc Loss=1.3613, Gen Loss=0.6810
Iteration 95400: Disc Loss=1.3550, Gen Loss=0.6854
Iteration 95500: Disc Loss=1.3630, Gen Loss=0.6710
Iteration 95600: Disc Loss=1.1213, Gen Loss=0.5490
Iteration 95700: Disc Loss=1.3719, Gen Loss=0.6625
Iteration 95800: Disc Loss=1.3774, Gen Loss=0.6935
Iteration 95900: Disc Loss=1.3682, Gen Loss=0.7225
Iteration 96000: Disc Loss=0.8097, Gen Loss=0.2986
[Checkpoint] Saved at iteration 96000
Iteration 96100: Disc Loss=1.3612, Gen Loss=0.5523
Iteration 96200: Disc Loss=1.3248, Gen Loss=0.6518
Iteration 96300: Disc Loss=0.8137, Gen Loss=0.4697
Iteration 96400: Disc Loss=1.3067, Gen Loss=0.6185
Iteration 96500: Disc Loss=1.3423, Gen Loss=0.5969
Iteration 96600: Disc Loss=0.9300, Gen Loss=0.3383
Iteration 96700: Disc Loss=1.3819, Gen Loss=0.3608
Ite